In [6]:
import numpy as np

class NeuralNetwork(object):
    def __init__(self, layers, random_seed):
        self.layers = layers
        self.random_seed = random_seed
        for i, layer in enumerate(self.layers):
            setattr(layer, 'random_seed', self.random_seed+i)
            layer.initialize_weights()

    def forwardpass(self, X):
        """ Calculate an output Y for the given input X. """
        X_next = X
        for layer in self.layers:
            X_next = layer.fprop(X_next)
        prediction = X_next
        return prediction

    def loss(self, prediction, Y):
        """ Calculate error on the given data. """
        loss = 0.5 * (Y - prediction) ** 2
        return -1.0 * (Y - prediction)

    def backpropogate(self, loss):
        """ Calculate an output Y for the given input X. """
        loss_next = loss
        for layer in reversed(self.layers):
            loss_next = layer.bprop(loss_next)
        return loss

class Layer(object):
    def _setup(self, input_shape):
        """ Setup layer with parameters that are unknown at __init__(). """
        pass

    def fprop(self, input):
        """ Calculate layer output for given input (forward propagation). """
        raise NotImplementedError()

    def bprop(self, output_grad):
        """ Calculate input gradient. """
        raise NotImplementedError()

# Break out activations as separate layers? As in Keras?
class Linear(Layer):

    random_seed = None

    def __init__(self, n_in, n_out,
                 activation_function):
        self.n_in = n_in
        self.n_out = n_out
        self.iteration = 0
        self.activation_function = activation_function

    def initialize_weights(self):
        np.random.seed(seed=self.random_seed)
        self.W = np.random.normal(size=(self.n_in, self.n_out))

    def fprop(self, layer_input):
        self.layer_input = layer_input
        self.activation_input = np.dot(layer_input, self.W)
        return self.activation_function(self.activation_input, bprop=False)

    def bprop(self, layer_gradient):
        dOutdActivationInput = self.activation_function(self.activation_input, bprop=True)
        dLayerInputdActivationInput = layer_gradient * dOutdActivationInput
        dActivationOutputdActivationInput = self.layer_input.T
        output_grad = np.dot(dLayerInputdActivationInput, self.W.T)
        weight_update = np.dot(dActivationOutputdActivationInput, dLayerInputdActivationInput)
        W_new = self.W - weight_update
        self.W = W_new
        self.iteration += 1
        return output_grad

def sigmoid(x, bprop=False):
    if bprop:
        s = sigmoid(x)
        return s*(1-s)
    else:
        return 1.0/(1.0+np.exp(-x))

# Not sure how I'm going to do this...
class TwoLayerNetwork(NeuralNetwork):
    '''
    Class for a neural network with only one hidden layer.
    '''
    def __init__(self, layers, random_seed):
        NeuralNetwork.__init__(self, layers, random_seed)

    def return_layer_outputs(self, X):
        """ Calculate an output Y for the given input X. """
        X_next = X
        layer_outputs = []
        for layer in self.layers:
            X_next = layer.fprop(X_next)
            # Collapse across observations
            np.mean(X_next, axis=0)
            layer_outputs.append(X_next)
        return layer_outputs

    def return_weights(self, X):
        """ Calculate an output Y for the given input X. """
        weights = [layer.W for layer in self.layers]
        return weights

In [7]:
layer1 = Linear(n_in=10, n_out=5, activation_function=sigmoid)
layer2 = Linear(n_in=5, n_out=1, activation_function=sigmoid)

nn = TwoLayerNetwork(layers=[layer1, layer2], random_seed=823)

nn

X = np.random.randn(10, 10)

first_weights = nn.return_weights(X)[0]

In [21]:
mi = np.amin(first_weights)
ma = np.amax(first_weights)
(first_weights - mi) / (ma - mi) + 0.5

array([[ 1.00644841,  0.88649247,  0.67753322,  1.01741548,  1.18971463],
       [ 1.27389194,  0.84260879,  0.88460243,  1.16664166,  0.89296368],
       [ 0.79651676,  1.18825594,  1.21733275,  0.86498883,  1.20110199],
       [ 0.9522418 ,  0.95899815,  1.04045733,  1.02524956,  1.06629017],
       [ 0.98521245,  1.49105541,  1.0447856 ,  0.76285711,  0.61836877],
       [ 1.23871817,  0.74253116,  0.5       ,  1.13712454,  1.5       ],
       [ 1.02340424,  0.96924283,  0.92467579,  0.84612442,  1.22983083],
       [ 1.05861927,  1.07481473,  1.36508367,  0.85542893,  0.67962392],
       [ 0.8870206 ,  0.76479964,  1.08442148,  0.98325262,  1.16943518],
       [ 0.63574116,  1.17146179,  1.00462037,  0.59158673,  0.98949977]])

In [12]:
np.amax(nn.return_weights(X)[0])

2.5969871334402539

In [37]:
np.mean(nn.return_layer_outputs(X)[0], axis=0)

array([ 0.52956117,  0.40004239,  0.58557413,  0.66468608,  0.52275286])

In [45]:
%matplotlib inline
import datetime
import random
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from matplotlib.dates import DateFormatter

fig=Figure()
ax=fig.add_subplot(111)
x=[]
y=[]
now=datetime.datetime.now()
delta=datetime.timedelta(days=1)
for i in range(10):
    x.append(now)
    now+=delta
    y.append(random.randint(0, 1000))
ax.plot_date(x, y, '-')
ax.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
fig.autofmt_xdate()
canvas=FigureCanvas(fig)

In [48]:
canvas.print_png('test.png')

In [7]:
from sklearn.datasets import make_classification
import numpy as np
import matplotlib

In [45]:
X1, Y1 = make_classification(n_features=3, 
                             n_redundant=0, 
                             n_informative=1,
                             n_clusters_per_class=1)

In [46]:
Y1.reshape(100,1).shape

(100, 1)

In [47]:
x_cols = ['X' + str(x) for x in range(3)]
all_cols = x_cols + ["Y"]
all_cols

['X0', 'X1', 'X2', 'Y']

In [48]:
df = pd.DataFrame(np.concatenate((X1, Y1.reshape(100,1)), axis=1), columns=all_cols)

In [49]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler(feature_range=(1,2))
df[x_cols] = mms.fit_transform(df[x_cols])
df.describe()

,X0,X1,X2,Y
count,100.000000,100.000000,100.000000,100.000000
mean,1.511731,1.344116,1.539174,0.500000
std,0.192096,0.293248,0.191122,0.502519
min,1.000000,1.000000,1.000000,0.000000
25%,1.365022,1.099991,1.398832,0.000000
50%,1.504772,1.193656,1.546733,0.500000
75%,1.627565,1.571031,1.669257,1.000000
max,2.000000,2.000000,2.000000,1.000000


In [8]:
layers = [[1.248, 1.073], [0.588, 0.63, 0.42], [0.187]]
coords = [{'x': 100.0, 'y': 100.0, 'layer': 0, 'neuron': 0, 'value': 1}, {'x': 100.0, 'y': 380.0, 'layer': 0, 'neuron': 1, 'value': 1}, {'x': 300.0, 'y': 50.0, 'layer': 1, 'neuron': 0, 'value': 1}, {'x': 300.0, 'y': 240.0, 'layer': 1, 'neuron': 1, 'value': 1}, {'x': 300.0, 'y': 430.0, 'layer': 1, 'neuron': 2, 'value': 1}, {'x': 500.0, 'y': 240.0, 'layer': 2, 'neuron': 0, 'value': 1}]

In [11]:
def update_coordinate_values(layers, coords):
    for i in range(len(layers)):
        layer = layers[i]
        for item in coords:
            for j, element in enumerate(layer):
                if item['layer'] == i and item['neuron'] == j:
                    item['value'] = element
    return coords
                
            

In [1]:
weights = {'input_to_hidden': [{'value': 1.0, 'coordinates': (0, 0)}, {'value': 1.0, 'coordinates': (0, 1)}, {'value': 1.0, 'coordinates': (0, 2)}, {'value': 1.0, 'coordinates': (0, 3)}, {'value': 1.0, 'coordinates': (1, 0)}, {'value': 1.0, 'coordinates': (1, 1)}, {'value': 1.0, 'coordinates': (1, 2)}, {'value': 1.0, 'coordinates': (1, 3)}, {'value': 1.0, 'coordinates': (2, 0)}, {'value': 1.0, 'coordinates': (2, 1)}, {'value': 1.0, 'coordinates': (2, 2)}, {'value': 1.0, 'coordinates': (2, 3)}], 'hidden_to_output': [{'value': 1.0, 'coordinates': (0, 0)}, {'value': 1.0, 'coordinates': (1, 0)}, {'value': 1.0, 'coordinates': (2, 0)}, {'value': 1.0, 'coordinates': (3, 0)}]}
weight_values = [[[1.0515391595736467, 0.8503918771900958, 0.5, 1.0699292256770176], [1.3588478604399405, 1.5, 0.7768058251726238, 0.8472225736044412], [1.3201581086628709, 0.8612430866281321, 0.699516726082643, 1.3564018749974078]], [[0.5], [0.8793001071496724], [0.7701922712827398], [1.5]]]

In [2]:
weight_values[0]

[[1.0515391595736467, 0.8503918771900958, 0.5, 1.0699292256770176],
 [1.3588478604399405, 1.5, 0.7768058251726238, 0.8472225736044412],
 [1.3201581086628709,
  0.8612430866281321,
  0.699516726082643,
  1.3564018749974078]]

In [3]:
weights_new = [0, 0]
weights_new[0] = weights['input_to_hidden']
weights_new[1] = weights['hidden_to_output']
weights_new

[[{'coordinates': (0, 0), 'value': 1.0},
  {'coordinates': (0, 1), 'value': 1.0},
  {'coordinates': (0, 2), 'value': 1.0},
  {'coordinates': (0, 3), 'value': 1.0},
  {'coordinates': (1, 0), 'value': 1.0},
  {'coordinates': (1, 1), 'value': 1.0},
  {'coordinates': (1, 2), 'value': 1.0},
  {'coordinates': (1, 3), 'value': 1.0},
  {'coordinates': (2, 0), 'value': 1.0},
  {'coordinates': (2, 1), 'value': 1.0},
  {'coordinates': (2, 2), 'value': 1.0},
  {'coordinates': (2, 3), 'value': 1.0}],
 [{'coordinates': (0, 0), 'value': 1.0},
  {'coordinates': (1, 0), 'value': 1.0},
  {'coordinates': (2, 0), 'value': 1.0},
  {'coordinates': (3, 0), 'value': 1.0}]]

In [5]:
weight_values

[[[1.0515391595736467, 0.8503918771900958, 0.5, 1.0699292256770176],
  [1.3588478604399405, 1.5, 0.7768058251726238, 0.8472225736044412],
  [1.3201581086628709,
   0.8612430866281321,
   0.699516726082643,
   1.3564018749974078]],
 [[0.5], [0.8793001071496724], [0.7701922712827398], [1.5]]]

In [13]:
def update_weight_values(weights, weight_values):
    for k in range(len(weights)):
        for i, weight_col in enumerate(weight_values[k]):
            for j, weight_val in enumerate(weight_col):
                for el in weights[k]:
#                     print(el)
                    if el['coordinates'][0] == i and el['coordinates'][1] == j:
                        el['value'] = weight_val
    return weights

In [14]:
update_weight_values(weights_new, weight_values)

[[{'coordinates': (0, 0), 'value': 1.0515391595736467},
  {'coordinates': (0, 1), 'value': 0.8503918771900958},
  {'coordinates': (0, 2), 'value': 0.5},
  {'coordinates': (0, 3), 'value': 1.0699292256770176},
  {'coordinates': (1, 0), 'value': 1.3588478604399405},
  {'coordinates': (1, 1), 'value': 1.5},
  {'coordinates': (1, 2), 'value': 0.7768058251726238},
  {'coordinates': (1, 3), 'value': 0.8472225736044412},
  {'coordinates': (2, 0), 'value': 1.3201581086628709},
  {'coordinates': (2, 1), 'value': 0.8612430866281321},
  {'coordinates': (2, 2), 'value': 0.699516726082643},
  {'coordinates': (2, 3), 'value': 1.3564018749974078}],
 [{'coordinates': (0, 0), 'value': 0.5},
  {'coordinates': (1, 0), 'value': 0.8793001071496724},
  {'coordinates': (2, 0), 'value': 0.7701922712827398},
  {'coordinates': (3, 0), 'value': 1.5}]]

In [16]:
def update_weight_values(weights, weight_values):weights_new

[[{'coordinates': (0, 0), 'value': 1.0},
  {'coordinates': (0, 1), 'value': 1.0},
  {'coordinates': (0, 2), 'value': 1.0},
  {'coordinates': (0, 3), 'value': 1.0},
  {'coordinates': (1, 0), 'value': 1.0},
  {'coordinates': (1, 1), 'value': 1.0},
  {'coordinates': (1, 2), 'value': 1.0},
  {'coordinates': (1, 3), 'value': 1.0},
  {'coordinates': (2, 0), 'value': 1.0},
  {'coordinates': (2, 1), 'value': 1.0},
  {'coordinates': (2, 2), 'value': 1.0},
  {'coordinates': (2, 3), 'value': 1.0}],
 [{'coordinates': (0, 0), 'value': 1.0},
  {'coordinates': (1, 0), 'value': 1.0},
  {'coordinates': (2, 0), 'value': 1.0},
  {'coordinates': (3, 0), 'value': 1.0}]]

In [1]:
def update_coordinate_values(layers, coords):
    for i in range(len(layers)):
        layer = layers[i]
        for item in coords:
            for j, element in enumerate(layer):
                if item['layer'] == i and item['neuron'] == j:
                    item['value'] = element
    return coords

In [3]:
layers = [[10.0, 20.0], [10.0, 20.0, 12.901], [15.0]]
coords = [[{'x': 100.0, 'y': 100.0, 'layer': 0, 'neuron': 0, 'value': 1}, {'x': 100.0, 'y': 380.0, 'layer': 0, 'neuron': 1, 'value': 1}], [{'x': 300.0, 'y': 50.0, 'layer': 1, 'neuron': 0, 'value': 1}, {'x': 300.0, 'y': 240.0, 'layer': 1, 'neuron': 1, 'value': 1}, {'x': 300.0, 'y': 430.0, 'layer': 1, 'neuron': 2, 'value': 1}], {'x': 500.0, 'y': 240.0, 'layer': 2, 'neuron': 0, 'value': 1}]

In [5]:
coords[0]

[{'layer': 0, 'neuron': 0, 'value': 1, 'x': 100.0, 'y': 100.0},
 {'layer': 0, 'neuron': 1, 'value': 1, 'x': 100.0, 'y': 380.0}]

In [4]:
update_coordinate_values(layers, coords)

TypeError: list indices must be integers or slices, not str

In [5]:
import numpy as np
a = np.float64(15.0)

In [10]:
type(a) == np.float64

True

## Shelve

In [1]:
import shelve

In [5]:
with shelve.open('spam') as db:
    db['eggs'] = 'eggs'

In [7]:
with shelve.open('spam') as db:
    print(db['eggs'])

eggs
